In [1]:
import pymysql
from selenium import webdriver
import time
import re
import sys
import pandas as pd
import urllib.request

In [2]:
# db connection
conn = pymysql.connect(host="ppam.ceubmhz1zgkv.ap-northeast-1.rds.amazonaws.com",user = "admin",password ="ppam1234", db="ppam_test",charset="utf8")
curs = conn.cursor(pymysql.cursors.DictCursor)


In [4]:
# review table 생성을 위한 쿼리문
sql ='''
        CREATE TABLE review_table2(
        review_code VARCHAR(15) NOT NULL,
        review_date VARCHAR(8) NOT NULL,
        review_raw VARCHAR(2000) NOT NULL,
        review_cleaned VARCHAR(2000),
        PRIMARY KEY(review_code))CHARSET = utf8;
    '''    
curs.execute(sql)
conn.commit()

OperationalError: (1050, "Table 'review_table2' already exists")

In [3]:
# post_table 불러오기
post_table = pd.read_sql("select * from post_table", conn)
post_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 281 entries, 0 to 280
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   post_code     281 non-null    object
 1   post_name     281 non-null    object
 2   post_url      281 non-null    object
 3   product_name  0 non-null      object
 4   size          0 non-null      object
 5   image_name    281 non-null    object
dtypes: object(6)
memory usage: 13.3+ KB


In [4]:
# 리뷰 크롤러 함수 정의
def review_crawling(post_url, post_code):
    # 리뷰가 없을 경우를 위한 flag
    flag = True
    
    # 상세페이지 접속
    driver = webdriver.Chrome(r"C:\Users\이다혜\Desktop\crawling_DB\chromedriver.exe")
    driver.get(post_url)
    time.sleep(5)

    # 상품평 버튼 보이게 스크롤바 내리기
    for i in range(200, 2000, 200):
        driver.execute_script('window.scrollTo(0,i)')
        try:
            # 상품평 버튼 클릭
            review_button = driver.find_element_by_css_selector('div#btfTab ul.tab-titles > li:nth-of-type(2)')
            review_button.click()
            time.sleep(10)

            # 최신순 보기 버튼 눌러지는지 확인
            new_review_button = driver.find_element_by_css_selector(
                'button.sdp-review__article__order__sort__newest-btn.js_reviewArticleNewListBtn.js_reviewArticleSortBtn')
            new_review_button.click()
            time.sleep(10)
            break
        except:
            pass

    # 베스트 리뷰 버튼 누르기
    try:
        best_review_button = driver.find_element_by_css_selector(
            'button.sdp-review__article__order__sort__best-btn.js_reviewArticleHelpfulListBtn.js_reviewArticleSortBtn')
        best_review_button.click()
        time.sleep(5)
    except:
        flag = False
        review_code_list = []
        review_date_list = []
        review_list = []
        
    # 리뷰가 있을 경우    
    if flag is True:
        # 베스트 리뷰 50개 크롤링
        review_code_list = []
        review_date_list = []
        review_list = []
        cnt = 1
        for i in range(1, 11):
            container = driver.find_elements_by_css_selector("section.js_reviewArticleListContainer article")
            for con in container:
                # 작성 날짜
                date = con.find_element_by_css_selector(
                    'div.sdp-review__article__list__info__product-info__reg-date').text.replace('.', '')
                # 리뷰 내용
                try:
                    review = con.find_element_by_css_selector(
                        'div.sdp-review__article__list__review.js_reviewArticleContentContainer').text.replace('\n', ' ')
                    review = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', '', review)
                    
                    # print(date + ',' + review + ',' + str(cnt))
                    review_code = post_code + '-' + str(cnt)
                    review_code_list.append(review_code)
                    review_date_list.append(date)
                    review_list.append(review)
                    
                    cnt += 1
                except:
                    pass
            try:
                page_button = driver.find_element_by_css_selector(
                    'div.sdp-review__article__page.js_reviewArticlePagingContainer > button:nth-child(' + str(i + 2) + ')')
                page_button.click()
                time.sleep(3)
            except:
                break
    
        
        # 최신순 보기 버튼 누르기
        new_review_button = driver.find_element_by_css_selector(
            'button.sdp-review__article__order__sort__newest-btn.js_reviewArticleNewListBtn.js_reviewArticleSortBtn')
        new_review_button.click()
        time.sleep(5)
        
        cnt_new = 1
        for i in range(1, 100):
            # 리뷰 컨테이너 선택
            container = driver.find_elements_by_css_selector("section.js_reviewArticleListContainer article")
            for con in container:
                try:
                    if cnt_new == 101:
                        print('100개 수집 완료')
                        break
                    # 작성 날짜
                    date = con.find_element_by_css_selector(
                        'div.sdp-review__article__list__info__product-info__reg-date').text.replace('.', '')
                    # 리뷰 내용
                    review = con.find_element_by_css_selector(
                        'div.sdp-review__article__list__review.js_reviewArticleContentContainer').text.replace('\n', ' ')
                    review = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', '', review)
                    # print(date + ',' + review + ',' + str(cnt_new))
                    
                    
                    review_code = post_code + '-' + str(cnt)
                    review_code_list.append(review_code)
                    review_date_list.append(date)
                    review_list.append(review)
                    cnt_new += 1
                    cnt += 1
                    
                except:
                    pass
                
            if cnt_new == 101:
                print('100개 수집 완료, 종료')
                break
            
            # 다음 페이지 버튼 있는지 확인
            try:
                page_button = driver.find_element_by_css_selector(
                    'div.sdp-review__article__page.js_reviewArticlePagingContainer > button:nth-child(' + str(i % 11 + 2) + ')')
                page_button.click()
                time.sleep(3)
            # 없어서 에러 나면 최신순 리뷰 수집 종료
            except:
                print('100개 미만 수집완료')
                break
        
    driver.close()
    return review_code_list, review_date_list, review_list  # 전체 리스트를 리턴
    

In [5]:
# 특수문자 제거하기 
def remove_specialChar(review_list):
    review_removed_specialchar_list = []
    for review in review_list:
        p = re.compile('[ 가-힣]')
        review = p.findall(review)
        review = ''.join(review)
        review_removed_specialchar_list.append(review)
    return review_removed_specialchar_list

# 띄어쓰기 및 맞춤법 수정
def review_spacing(review_list):
    review_cleaned_list = []
    for review in review_list:
    # review가 null값일 경우
        try:
            review_cleaned_list.append(spacing(review))
        except:
            review_cleaned_list.append(review)
    return review_cleaned_list

In [6]:
# db에 넣기
def insert_to_db(review_code_list, review_date_list, review_list):
    review_removed_specialchar_list = remove_specialChar(review_list)
    review_cleaned_list = review_spacing(review_removed_specialchar_list)
    for i in range(len(review_code_list)):
        sql ='''INSERT INTO ppam_test.review_table2 (review_code, review_date, review_raw, review_cleaned) 
                VALUES(%s,%s,%s,%s);'''
        val = (review_code_list[i], review_date_list[i], review_list[i], review_cleaned_list[i]) 
        curs.execute(sql,val)
        conn.commit()

In [7]:
# for문 돌면서 해당 url에 접속해서 리뷰 가져오기
for i in range(2):
    review_code_list, review_date_list, review_list = review_crawling(post_table['post_url'][i], post_table['post_code'][i])
    print(post_table['post_code'][i], len(review_code_list))
    # 게시글 리뷰 수가 30개 이상일 경우만 DB에 넣는다
    if len(review_code_list) >= 30:
        insert_to_db(review_code_list, review_date_list, review_list)
    else:
        pass

100개 미만 수집완료
A1-1 42
100개 미만 수집완료
A1-2 23
